In [ ]:
pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

In [1]:
pip install gdown

  Installing build dependencies ... - error
  ERROR: Command errored out with exit status 1:
   command: /anaconda/envs/azureml_py38/bin/python /anaconda/envs/azureml_py38/lib/python3.8/site-packages/pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-5yht2dv5/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'setuptools>=40.8.0' wheel
       cwd: None
  Complete output (14 lines):
  Traceback (most recent call last):
    File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 193, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pip/__main__.py", line 23, in <module>
      from pip._internal.cli.main import main as _main  # isort:skip # noqa
    File "/anaconda/envs/azureml_py38/lib/python3.8/site-package

In [35]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd

In [36]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9f1b1678-0b39-4d15-98d4-6e2f53560eca'
resource_group = 'Test_DSPD'
workspace_name = 'DSPD_ml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='price_ds')
dataset.download(target_path='./price', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/tesladspd/code/Users/m.zaib.23237/price/price.csv']

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10
np.random.seed(42)

In [2]:
pip install boxx

     |████████████████████████████████| 85 kB 273 kB/s  eta 0:00:01
     |████████████████████████████████| 282 kB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 66.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 1.3 MB/s  eta 0:00:01
  Created wheel for boxx: filename=boxx-0.9.3-py3-none-any.whl size=91364 sha256=9f7241718a80133e1b4b623c556b482778c1492e360c9763907b0ccbee7a2923
  Stored in directory: /home/azureuser/.cache/pip/wheels/8b/f9/55/37d470bf2f4bb95b13d6907d20c8a79f37571bae49c166173f
  Created wheel for py-heat: filename=py_heat-0.0.6-py2.py3-none-any.whl size=7801 sha256=0d54b12fa13bd38033a228d0426b1826456c691d4462f982772039623506921a
  Stored in directory: /home/azureuser/.cache/pip/wheels/77/c0/6f/8704e1590c6ad1a7ed3dc3b5ee8ca81e2151d6f4ce2288ff73
  Created wheel for fn: filename=fn-0.4.3-py3-none-any.whl size=28473 sha256=afd92696ab82e79b113da7be306882ec85fd9477a4b0b626bec145ce803bbc12
  Stored in directory: /home/azureuser/.cache/pip

In [6]:
import boxx  
train_js = boxx.loadjson('dataset/instances_train2019.json')
#cat_list=[1,2,3,4,5,6,8,7,9,10,11,12,13,14,15,16,17,18,20,19]
#Users/m.zafar/data-Science-project-master/master/instances_train2019.json

In [7]:
categories_df = pd.DataFrame(train_js['categories'])
annotations_df = pd.DataFrame(train_js['annotations'])
annotations_df=annotations_df.rename(columns={'id':'annotation_id'})
images_df = pd.DataFrame(train_js['images'])
images_df=images_df.rename(columns={'id':'image_id'})
categories_df=categories_df.rename(columns={'id':'category_id'})
temp=pd.merge(annotations_df,images_df,how="inner",on="image_id")
train=pd.merge(temp,categories_df,how="left",on="category_id")

In [18]:
price_df = pd.read_csv('price/price.csv')
cat_list = []
#price_df['Product_name']
cat_list = list(categories_df.loc[categories_df['name'].isin(price_df['Product_name']),['category_id']]['category_id'])

In [19]:
#train = train.loc[train['category_id']<21]
train = train.loc[train['category_id'].isin(cat_list)]

In [20]:
len(train)

640

In [21]:
import shutil
import os

src_dir = "dataset/train2019/"#'OneDrive - Institute of Business Administration/archive/images/train2019'
dst_dir = "master/images/train/"
shutil.rmtree(dst_dir)
for jpgfile in train['file_name']:
    src_path = os.path.join(src_dir,jpgfile)
    if not os.path.exists(dst_dir): os.makedirs(dst_dir)
    #print(src_path)
    #print(jpgfile)
    shutil.copy(src_path, dst_dir)
    #print(src_path)

In [22]:
import os

APP_FOLDER = 'master/images/train'

totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1


print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Searching in :  master/images/train
Total number of files 640
Total Number of directories 0
Total: 640


In [23]:
import boxx  
val_js = boxx.loadjson('dataset/instances_val2019.json')

In [24]:
categories_df_val = pd.DataFrame(val_js['categories'])
annotations_df_val = pd.DataFrame(val_js['annotations'])
images_df_val = pd.DataFrame(val_js['images'])
images_df_val=images_df_val.rename(columns={'id':'image_id'})
categories_df_val=categories_df_val.rename(columns={'id':'category_id'})
temp=pd.merge(annotations_df_val,images_df_val,how="inner",on="image_id")
val=pd.merge(temp,categories_df_val,how="left",on="category_id")

In [25]:
#val = val.loc[val['category_id']<21]
val = val.loc[val['category_id'].isin(cat_list)]


In [26]:
import os

APP_FOLDER = 'master/images/val/'

totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1


print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

len(val)

Searching in :  master/images/val/
Total number of files 2972
Total Number of directories 0
Total: 2972


673

In [27]:
len(val['file_name'].unique())

326

In [28]:
import shutil
import os

src_dir = "dataset/val2019/"#'OneDrive - Institute of Business Administration/archive/images/train2019'
dst_dir = "master/images/val/"
shutil.rmtree(dst_dir)
for jpgfile in set(val['file_name'].unique()):
    src_path = os.path.join(src_dir,jpgfile)
    if not os.path.exists(dst_dir): os.makedirs(dst_dir)
    #print(src_path)
    #print(jpgfile)
    shutil.copy(src_path, dst_dir)
    #print(src_path)

In [15]:
(set(val['name'])) #list of all labels in val dataset

{'10_puffed_food',
 '11_puffed_food',
 '12_puffed_food',
 '13_dried_fruit',
 '14_dried_fruit',
 '15_dried_fruit',
 '16_dried_fruit',
 '17_dried_fruit',
 '18_dried_fruit',
 '19_dried_fruit',
 '1_puffed_food',
 '20_dried_fruit',
 '2_puffed_food',
 '3_puffed_food',
 '4_puffed_food',
 '5_puffed_food',
 '6_puffed_food',
 '7_puffed_food',
 '8_puffed_food',
 '9_puffed_food'}

In [29]:
def create_dataset(dataset_type, dtf):
  #print(dtf[0:1])
  images_path = Path(f"master/images/{dataset_type}")
  images_path.mkdir(parents=True, exist_ok=True)
  labels_path = Path(f"master/labels/{dataset_type}")
  labels_path.mkdir(parents=True, exist_ok=True)
  for idx,row in dtf.iterrows():
    #print(row)
    label_name = f"{row['file_name'].split('.')[0]}.txt"
    with (labels_path / label_name).open(mode="a") as label_file:
        category_idx = row['category_id']
        label_file.write(
           f"{category_idx} {(row['point_xy'][0])/row['width']} {(row['point_xy'][1])/row['height']} {(row['bbox'][2])/row['width']} {(row['bbox'][3])/row['height']}\n"
        )

In [18]:
train.describe()

,area,category_id,annotation_id,image_id,iscrowd,width,height
count,6.398000e+03,6398.000000,6398.000000,6398.000000,6398.0,6398.0,6398.0
mean,6.235223e+05,10.498906,22177.131604,22177.131604,0.0,2592.0,1944.0
std,2.564445e+05,5.767274,15360.482459,15360.482459,0.0,0.0,0.0
min,1.141877e+05,1.000000,3517.000000,3517.000000,0.0,2592.0,1944.0
25%,4.264481e+05,5.250000,9912.250000,9912.250000,0.0,2592.0,1944.0
50%,5.858317e+05,10.000000,12786.500000,12786.500000,0.0,2592.0,1944.0
75%,8.108593e+05,15.750000,30383.750000,30383.750000,0.0,2592.0,1944.0
max,1.361675e+06,20.000000,50381.000000,50381.000000,0.0,2592.0,1944.0


In [19]:
val.describe()

,area,category_id,id,image_id,iscrowd,width,height
count,7776.000000,7776.000000,7776.000000,7776.00000,7776.0,7776.000000,7776.000000
mean,310044.583997,10.567901,193188.444187,18952.50823,0.0,1829.010159,1829.010159
std,75150.644851,5.536760,105876.925963,7975.49677,0.0,21.261780,21.261780
min,73252.670000,1.000000,889.000000,254.00000,0.0,1751.000000,1751.000000
25%,251968.270000,6.000000,104476.000000,13200.75000,0.0,1816.000000,1816.000000
50%,301643.430000,10.000000,201240.500000,20566.50000,0.0,1832.000000,1832.000000
75%,362456.747500,16.000000,277743.500000,25354.75000,0.0,1845.000000,1845.000000
max,631238.390000,20.000000,376022.000000,30616.00000,0.0,1886.000000,1886.000000


In [30]:
dic = {}
cat_list.sort()
for ind,catid in enumerate(cat_list):
    dic[ind] = categories_df.loc[categories_df['category_id']==catid]['name'].item()
label_name_list = []
for cat in cat_list:
    label_name_list.append(categories_df.loc[categories_df['category_id']==cat]['name'].item())
    train.loc[train['name'] == categories_df.loc[categories_df['category_id']==cat]['name'].item(), 'category_id' ] = list(dic.keys())[list(dic.values()).index(categories_df.loc[categories_df['category_id']==cat]['name'].item())]
    val.loc[val['name'] == categories_df.loc[categories_df['category_id']==cat]['name'].item(), 'category_id' ] = list(dic.keys())[list(dic.values()).index(categories_df.loc[categories_df['category_id']==cat]['name'].item())]

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':


In [31]:
num_categories = len(cat_list)

In [32]:
cat_name_list = list(dic.values())
cat_name_list

['1_puffed_food', '2_puffed_food']

In [30]:
type(list(dic.values()))

list

In [22]:
train.describe()

,area,category_id,annotation_id,image_id,iscrowd,width,height
count,6.398000e+03,6398.000000,6398.000000,6398.000000,6398.0,6398.0,6398.0
mean,6.235223e+05,9.498906,22177.131604,22177.131604,0.0,2592.0,1944.0
std,2.564445e+05,5.767274,15360.482459,15360.482459,0.0,0.0,0.0
min,1.141877e+05,0.000000,3517.000000,3517.000000,0.0,2592.0,1944.0
25%,4.264481e+05,4.250000,9912.250000,9912.250000,0.0,2592.0,1944.0
50%,5.858317e+05,9.000000,12786.500000,12786.500000,0.0,2592.0,1944.0
75%,8.108593e+05,14.750000,30383.750000,30383.750000,0.0,2592.0,1944.0
max,1.361675e+06,19.000000,50381.000000,50381.000000,0.0,2592.0,1944.0


In [23]:
val.describe()

,area,category_id,id,image_id,iscrowd,width,height
count,7776.000000,7776.000000,7776.000000,7776.00000,7776.0,7776.000000,7776.000000
mean,310044.583997,9.567901,193188.444187,18952.50823,0.0,1829.010159,1829.010159
std,75150.644851,5.536760,105876.925963,7975.49677,0.0,21.261780,21.261780
min,73252.670000,0.000000,889.000000,254.00000,0.0,1751.000000,1751.000000
25%,251968.270000,5.000000,104476.000000,13200.75000,0.0,1816.000000,1816.000000
50%,301643.430000,9.000000,201240.500000,20566.50000,0.0,1832.000000,1832.000000
75%,362456.747500,15.000000,277743.500000,25354.75000,0.0,1845.000000,1845.000000
max,631238.390000,19.000000,376022.000000,30616.00000,0.0,1886.000000,1886.000000


In [35]:
create_dataset( 'train', train)
create_dataset('val', val)

In [33]:
import yaml

fname = "master/yolov5/data/rpc.yaml"

stream = open(fname, 'r')
data = yaml.load(stream)

data['nc']=len(cat_list) #Do remember to normalize cat_list
data['names'] = list(dic.values())
#data['instances'][0]['host'] = '1.2.3.4'
#data['instances'][0]['username'] = 'Username'
#data['instances'][0]['password'] = 'Password'

with open(fname, 'w') as yaml_file:
    yaml_file.write( yaml.dump(data, default_flow_style=False))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [34]:
fname = "master/yolov5/models/yolov5l.yaml"

stream = open(fname, 'r')
data = yaml.load(stream)

data['nc']=len(cat_list) #Do remember to normalize cat_list
#data['names'] = list(dic.values())
#data['instances'][0]['host'] = '1.2.3.4'
#data['instances'][0]['username'] = 'Username'
#data['instances'][0]['password'] = 'Password'

with open(fname, 'w') as yaml_file:
    yaml_file.write( yaml.dump(data, default_flow_style=False))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [29]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/dspd/code/Users/m.zafar'

In [ ]:
#code to be run for training on terminal

#python train.py --img 640 --batch 4 --epochs 1 --data ./data/rpc.yaml --cfg ./models/yolov5l.yaml --weights yolov5l.pt --name yolov5l_rpc --cache

Below code is just for practice

In [ ]:
cat_list=[1,3,5,7,9]


In [ ]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9f1b1678-0b39-4d15-98d4-6e2f53560eca'
resource_group = 'Test_DSPD'
workspace_name = 'DSPD_ml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='dataset_org')
dataset.download(target_path='.', overwrite=False)

https://www.howtogeek.com/407944/how-to-create-a-flow-from-scratch/